In [ ]:
import pandas as pd
import re
from datasets import Dataset, Audio

In [ ]:
dataset = "./ABBYY_clear.txt"

In [ ]:
with open(dataset, "r") as f: # reading a txt file
    data = f.readlines()

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df[['word', 'description']] = df[0].str.extract(r'^([^(=\[\-,]+?)(?:\s*[\(=\[\-,]\s*(.*))?$', expand=True)
df = df.drop(0, axis=1)

In [ ]:
# Cleaning
#########################
df.drop_duplicates(inplace=True)
df = df[~df["word"].str.contains(r'-\d+-', regex=True, na=False)] # -number- deletion
df['description'] = df['description'].fillna('N/A')
#########################

In [ ]:
df = df.dropna(subset=['word']) # NaN for word section

In [ ]:
df[df['description'] == 'N/A'].shape[0] # 2044 left, probably to drop, most of them are just page initials
df.drop(df[df['description'] == 'N/A'].index, inplace=True)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("ZiartisNikolas/NMT-cypriot-dialect-to-greek")
model = AutoModelForSeq2SeqLM.from_pretrained("ZiartisNikolas/NMT-cypriot-dialect-to-greek")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Если используете несколько GPU, включите DataParallel
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)

In [ ]:
# !pip install sacremoses

In [ ]:
df.info()

In [ ]:
complete = pd.read_csv('full8-12.csv')
really_full = pd.read_csv('complete_dataset.csv')

In [ ]:
def translate_to_greek(text):
    if pd.isna(text) or text == '':
        return ''

    # Токенизация и перенос на GPU
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        # Исправление: убрать .module если не используете DataParallel
        if torch.cuda.device_count() > 1:
            outputs = model.module.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
        else:
            outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)

    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

In [ ]:
tqdm.pandas(desc="Translating")

# df['greek_word'] = df['word'][15000:].progress_apply(translate_to_greek)
complete['greek_description'] = complete['description'][8000:12000].progress_apply(translate_to_greek)

# print(df[['word', 'description', 'greek_translation']].head())

In [ ]:
complete.to_csv('full8-12.csv')

In [ ]:
complete.drop()

In [ ]:
complete[8050:8100]

In [ ]:
ready.info()

In [ ]:
# Вместо concat, просто заполним пропуски
ready = really_full.copy()

# Заполнить пропуски в greek_description данными из complete
ready.loc[8000:11999, 'greek_description'] = complete['greek_description'][8000:12000].values

print("Пропусков после заполнения:", ready['greek_description'].isna().sum())

In [ ]:
ready.info()

In [ ]:
ready.drop_duplicates()

In [ ]:
ready.info()

In [ ]:
ready = ready.drop(columns=['Unnamed: 0'], errors='ignore')

# Удалить дубликаты
ready = ready.drop_duplicates()

In [ ]:
ready = ready.dropna()

In [ ]:
ready.info()

In [ ]:
ready.to_csv('DATASET.csv')

In [ ]:
ready.head()

In [ ]:
ready = ready[['word', 'description', 'greek_word', 'greek_description']]

In [ ]:
ready

In [ ]:
#### Step 1
word_lengths = ready['word'].str.len()
desc_lengths = ready['description'].str.len()
shorter_desc = desc_lengths < word_lengths

print(f"Строк для удаления (description короче word): {shorter_desc.sum()}")

# Удаляем эти строки
ready = ready[~shorter_desc]

print(f"После удаления: {len(ready)}")
print(f"Удалено строк: {shorter_desc.sum()}")

In [ ]:
#############
# Step 2
print(f"Было строк: {len(ready)}")

# Пересчитываем для текущего датасета
has_digits = ready['word'].str.contains(r'\d', na=False)

print(f"Строк для удаления (word содержит цифры): {has_digits.sum()}")

# Удаляем эти строки
ready = ready[~has_digits]

print(f"После удаления: {len(ready)}")
print(f"Удалено строк: {has_digits.sum()}")

In [ ]:
# Step 3
print(f"Было строк: {len(ready)}")

repeated_chars_word = ready['greek_word'].str.contains(r'(.)\1{3,}', na=False)
repeated_chars_desc = ready['greek_description'].str.contains(r'(.)\1{3,}', na=False)

print(f"Строк с повторениями в greek_word: {repeated_chars_word.sum()}")
print(f"Строк с повторениями в greek_description: {repeated_chars_desc.sum()}")


to_remove = repeated_chars_word | repeated_chars_desc
print(f"Строк для удаления (повторения в greek_word ИЛИ greek_description): {to_remove.sum()}")


ready = ready[~to_remove]

print(f"После удаления: {len(ready)}")
print(f"Удалено строк: {to_remove.sum()}")

In [ ]:
##########
# Step 4
print(f"Было строк: {len(ready)}")
single_char_words = ready['word'].str.len() == 1

print(f"Строк для удаления (односимвольные слова): {single_char_words.sum()}")

ready = ready[~single_char_words]

print(f"После удаления: {len(ready)}")
print(f"Удалено строк: {single_char_words.sum()}")

In [ ]:
#########
# Step 5
print(f"Было строк: {len(ready)}")

digit_count = ready['description'].str.count(r'\d').fillna(0)
total_count = ready['description'].str.len().fillna(0)
non_digit_count = total_count - digit_count

more_digits = digit_count > non_digit_count

print(f"Строк для удаления (в description цифр больше чем остальных): {more_digits.sum()}")

ready = ready[~more_digits]

print(f"После удаления: {len(ready)}")
print(f"Удалено строк: {more_digits.sum()}")

In [ ]:
# single_char_words = ready['word'].str.len() == 1
# count_single_char = single_char_words.sum()

# has_digits = ready['word'].str.contains(r'\d', na=False)
# count_digits = has_digits.sum()

# repeated_chars_word = ready['greek_word'].str.contains(r'(.)\1{3,}', na=False)
# repeated_chars_desc = ready['greek_description'].str.contains(r'(.)\1{3,}', na=False)
# count_repeated_word = repeated_chars_word.sum()
# count_repeated_desc = repeated_chars_desc.sum()
# both_repeated = repeated_chars_word & repeated_chars_desc

# digit_count = ready['description'].str.count(r'\d').fillna(0)
# total_count = ready['description'].str.len().fillna(0)
# non_digit_count = total_count - digit_count

# # Найти строки где цифр больше чем остальных символов
# more_digits = digit_count > non_digit_count
# count_more_digits = more_digits.sum()

In [ ]:
ready.info()

In [ ]:
ready.to_csv('DATASET_CLEARED.csv')

In [ ]:
hf_df = Dataset.from_dict(ready)

In [ ]:
hf_df

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Shuffling data and make it work for HF
from datasets import Dataset, DatasetDict
import pandas as pd

ready_shuffled = ready.sample(frac=1, random_state=42).reset_index(drop=True)

shuffled_dict = {
    'word': ready_shuffled['word'].tolist(),
    'description': ready_shuffled['description'].tolist(),
    'greek_word': ready_shuffled['greek_word'].tolist(),
    'greek_description': ready_shuffled['greek_description'].tolist()
}

In [ ]:
# Количество строк одинаковое?
print(f"Строк до: {len(ready)}")
print(f"Строк после: {len(ready_shuffled)}")

# Содержат ли одинаковые данные (но в другом порядке)?
print(f"Одинаковые слова: {set(ready['word']) == set(ready_shuffled['word'])}")
print(f"Одинаковые описания: {len(set(ready['description'])) == len(set(ready_shuffled['description']))}")

In [ ]:
ready_shuffled.info()

In [ ]:
hf_shuffled = Dataset.from_dict(ready_shuffled)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
hf_shuffled

In [ ]:
train_temp = hf_shuffled.train_test_split(test_size=0.2, seed=42)
val_test = train_temp['test'].train_test_split(test_size=0.5, seed=42)

In [ ]:
final_dataset = DatasetDict({
    'train': train_temp['train'],        # 80% (~21,600)
    'validation': val_test['train'],     # 10% (~2,700)
    'test': val_test['test']            # 10% (~2,700)
})

In [ ]:
final_dataset

In [ ]:
final_dataset.push_to_hub(
   "Elormiden/cypriot-greek-dictionary",
   private=True
)